###  Countries Data

The table country_codes.csv contains a list of countries and their numeric, two and three letters country codes. Also it includes latitude and longitude coordinates of the geographic center of each country. This table will be useful later in the week when visualizing country level data on a map.

- Import the csv file as a pandas data frame.
- With Python, define a countries table in the climate database:
- Load the data frame into the countries table

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text

In [3]:
countries = pd.read_csv('../data/country_codes.csv')

In [4]:
countries

,name,alpha2,alpha3,code,lat,lon
0,Afghanistan,AF,AFG,4,33.0000,65.0
1,Albania,AL,ALB,8,41.0000,20.0
2,Algeria,DZ,DZA,12,28.0000,3.0
3,American Samoa,AS,ASM,16,-14.3333,-170.0
4,Andorra,AD,AND,20,42.5000,1.6
...,...,...,...,...,...,...
238,Wallis and Futuna,WF,WLF,876,-13.3000,-176.2
239,Western Sahara,EH,ESH,732,24.5000,-13.0
240,Yemen,YE,YEM,887,15.0000,48.0
241,Zambia,ZM,ZMB,894,-15.0000,30.0


In [5]:
from dotenv import dotenv_values

config = dotenv_values(".env")


In [6]:
# define variables for the login
username = config['USER']
password = config['PASS']
host = config['HOST']
port = config['PORT']

In [7]:
url = f'postgresql://{username}:{password}@{host}:{port}/climate'

In [8]:
engine = create_engine(url, echo=False)

In [9]:
with engine.begin() as conn:
    conn.execute(text("DROP TABLE IF EXISTS countries CASCADE;"))
    conn.execute(text("""
        CREATE TABLE countries (
            alpha2 VARCHAR(2) PRIMARY KEY,
            name VARCHAR,
            alpha3 VARCHAR(3),
            code int,
            lat NUMERIC,
            lon NUMERIC
        );
    """))
countries.to_sql('countries', engine, if_exists='append', index=False)

243

### Stations Data

Using the stations file found in the downloaded ECA_blend data folder proceed with it in the same way as for the countries data from the previous exercise. Use the downloaded stations file as it will have all stations found in the downloaded datasets.

Hints:

- First read stations data into a notebook and clean up before uploading to database
- The pd.read_csv method has a skiprows parameter to skip some header lines of a .csv file
- You need to cleanup the column names of the file. Watch out for whitespace and convert the names to lowercase
- Add a foreign key constraint for the cn column and let it point to the alpha2 column of the countries table

In [11]:
stations = pd.read_csv('../data/stations.txt', delimiter=',',skiprows=17)

In [12]:
stations

,STAID,STANAME,CN,LAT,LON,HGHT
0,1,VAEXJOE,SE,+56:52:00,+014:48:00,166
1,2,FALUN,SE,+60:37:00,+015:37:00,160
2,3,STENSELE,SE,+65:04:00,+017:09:59,325
3,4,LINKOEPING,SE,+58:24:00,+015:31:59,93
4,5,LINKOEPING-MALMSLAETT,SE,+58:24:00,+015:31:59,93
...,...,...,...,...,...,...
7060,26304,E39 UALAND,NO,+58:32:00,+006:19:32,213
7061,26305,E16 HALLINGBY,NO,+60:16:23,+010:11:11,159
7062,26306,E6 DRIVSTUA,NO,+62:25:37,+009:37:22,697
7063,26307,FV520 SVANDALSFOSSEN,NO,+59:37:28,+006:17:30,18


In [13]:
stations.columns = stations.columns.str.strip().str.lower()

In [14]:
stations

,staid,staname,cn,lat,lon,hght
0,1,VAEXJOE,SE,+56:52:00,+014:48:00,166
1,2,FALUN,SE,+60:37:00,+015:37:00,160
2,3,STENSELE,SE,+65:04:00,+017:09:59,325
3,4,LINKOEPING,SE,+58:24:00,+015:31:59,93
4,5,LINKOEPING-MALMSLAETT,SE,+58:24:00,+015:31:59,93
...,...,...,...,...,...,...
7060,26304,E39 UALAND,NO,+58:32:00,+006:19:32,213
7061,26305,E16 HALLINGBY,NO,+60:16:23,+010:11:11,159
7062,26306,E6 DRIVSTUA,NO,+62:25:37,+009:37:22,697
7063,26307,FV520 SVANDALSFOSSEN,NO,+59:37:28,+006:17:30,18


In [17]:
engine = create_engine(url, echo=False)
with engine.begin() as conn:
    conn.execute(text("DROP TABLE IF EXISTS stations CASCADE;"))
    conn.execute(text("""
        CREATE TABLE stations (
            staid INT PRIMARY KEY,
            staname VARCHAR,
            cn VARCHAR(2),
            lat VARCHAR,
            lon VARCHAR,
            hght INT,
            foreign key (cn) references countries(alpha2)
        );       
    """))
stations.to_sql('stations', engine, if_exists='append', index=False)

65

#### Add two new columns called latitude and longitude which will be updated versions of lat and long that we already have in our table:

In [18]:
with engine.begin() as conn:
    conn.execute(text("""
        ALTER TABLE stations
        ADD latitude NUMERIC;
        ALTER TABLE stations 
        ADD longitude NUMERIC;
        """))

####  Extract the latitude in a decimal form 

In [19]:
with engine.begin() as conn:
    conn.execute(text("""
    UPDATE stations SET latitude = (
    split_part(lat, ':', 1)::numeric + -- the degrees
    split_part(lat, ':', 2)::numeric/60+ -- the minutes divided by 60
    split_part(lat, ':', 3)::numeric/(60*60) -- the seconds divided by 3600 all summed up 
);
""" ))

#### Extract the latitude or longitude in a decimal form 

In [20]:
with engine.begin() as conn:
    conn.execute(text("""
    UPDATE stations SET longitude = (
    split_part(lon, ':', 1)::numeric +
    split_part(lon, ':', 2)::numeric/60+
    split_part(lon, ':', 3)::numeric/(60*60)
);
""" ))

In [21]:
stations

,staid,staname,cn,lat,lon,hght
0,1,VAEXJOE,SE,+56:52:00,+014:48:00,166
1,2,FALUN,SE,+60:37:00,+015:37:00,160
2,3,STENSELE,SE,+65:04:00,+017:09:59,325
3,4,LINKOEPING,SE,+58:24:00,+015:31:59,93
4,5,LINKOEPING-MALMSLAETT,SE,+58:24:00,+015:31:59,93
...,...,...,...,...,...,...
7060,26304,E39 UALAND,NO,+58:32:00,+006:19:32,213
7061,26305,E16 HALLINGBY,NO,+60:16:23,+010:11:11,159
7062,26306,E6 DRIVSTUA,NO,+62:25:37,+009:37:22,697
7063,26307,FV520 SVANDALSFOSSEN,NO,+59:37:28,+006:17:30,18


## Solve SQL Challeneges

#### 1. How many records are there in the temperature data?

In [22]:
with engine.begin() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM mean_temperature;"))
    record_count = result.scalar() #fetches the single value returned by the query

print(f"The temperature data contains {record_count} records.")

The temperature data contains 86436449 records.


#### 2. Get a list of all countries included. Remove all duplicates and sort it alphabetically.

In [23]:
with engine.begin() as conn:
    result = conn.execute(text("SELECT DISTINCT name FROM countries ORDER BY name ASC;"))
    countries = result.all()
countries_df = pd.DataFrame(countries,  columns = ['country'])

In [24]:
countries_df

,country
0,Afghanistan
1,Albania
2,Algeria
3,American Samoa
4,Andorra
...,...
238,Wallis and Futuna
239,Western Sahara
240,Yemen
241,Zambia


#### 3.  Get the number of weather stations for each country. Group by the number of stations in descending order!

In [25]:
with engine.begin() as conn:
    results = conn.execute(text("""
    SELECT c.name AS Name, 
    s.cn AS CN, 
    COUNT(*) AS Number_of_Stations
    FROM stations s
    JOIN countries c 
    ON c.alpha2 = s.cn
    GROUP BY s.cn, c.name
    ORDER BY Number_of_Stations DESC
    """))
    Number_of_stations = results.all()
stations_df = pd.DataFrame(Number_of_stations, columns=['Name', 'CN', 'Number_of_Stations'])

In [26]:
stations_df

,Name,CN,Number_of_Stations
0,Norway,NO,1467
1,Germany,DE,1079
2,Sweden,SE,854
3,Austria,AT,767
4,Italy,IT,663
5,Russia,RU,524
6,Finland,FI,410
7,Poland,PL,338
8,Spain,ES,209
9,United Kingdom,GB,120


#### 4. What’s the average height of stations in Switzerland compared to Netherlands?

In [36]:
with engine.begin() as conn:
    results = conn.execute(text("""
    SELECT c.name AS Country_Name, 
    s.cn AS Country_Code, 
    AVG(s.hght) AS Average_Height
    FROM stations s
    JOIN countries c 
    ON c.alpha2 = s.cn
    WHERE s.cn IN ('CH', 'NL')
    GROUP BY s.cn, c.name
    """))
    avg_height = results.all()

avg_height_df = pd.DataFrame(avg_height, columns=['Country_Name', 'Country_Code', 'Average_Height'])

In [37]:
avg_height_df

,Country_Name,Country_Code,Average_Height
0,Switzerland,CH,845.7000000000000000
1,Netherlands,NL,15.8392857142857143


#### 5. What is the highest station in Germany?

In [38]:
with engine.begin() as conn:
    results = conn.execute(text("""
    SELECT staname, hght
    FROM stations
    WHERE cn = 'DE'
    ORDER BY hght DESC
    LIMIT 1
    """ ))
    highest_st = results.all()
highest_st_df = pd.DataFrame(highest_st, columns=['Station_Name', 'Height'])

In [39]:
highest_st_df

,Station_Name,Height
0,ZUGSPITZE,2964


#### 6. What’s the minimum and maximum daily average temperature ever recorded in Germany?

In [40]:
with engine.begin() as conn:
    results = conn.execute(text("""
    SELECT s.staname,
    MIN(t.tg) AS Min_temp, 
    MAX(t.tg) AS Max_temp
    FROM mean_temperature t
    INNER JOIN stations s 
    ON s.staid = t.staid
    WHERE s.cn = 'DE'
    GROUP BY s.staname
    """ ))
    temp = results.all()
temp_df = pd.DataFrame(temp, columns=['Station_Name', 'Min_temp','Max_temp'])    

In [41]:
temp_df

,Station_Name,Min_temp,Max_temp
0,AACH,-198,289
1,AACHEN,-158,290
2,AACHEN-ORSBACH,-108,303
3,AHAUS,-107,299
4,AHRENSBURG-WULFSDORF,-170,282
...,...,...,...
1072,ZINNOWITZ,-178,293
1073,ZINNWALD-GEORGENFELD,-226,262
1074,ZUGSPITZE,-331,136
1075,ZWIESEL,-228,264


In [42]:
temp_df['Min_temp'].min()

-331

In [43]:
temp_df['Max_temp'].max()

324